# <font color = black>Webscrapping</font>

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import bs4 as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [ ]:
#the function returns the url address from the first page to the 17th page
def get_url_list(page1_url):
    var_url = '&section_offset=5&items_offset='
    url_lst = [page1_url]
    for page in range(1,17):
        url = page1_url+var_url+str(18*page)
        url_lst.append(url)
    return url_lst

In [ ]:
#this function scraps the information for a specific apartment page
#the price information is only retrivable in the main page therefore we need to pass the price to this function

def get_house_info(url, price):
    #url = 'https://www.airbnb.com/rooms/3503946?s=P8Cyps_7'
    import requests
    from bs4 import BeautifulSoup
    import re
    response = requests.get(url)
    output = []
    if not response.status_code == 200:
        print('failed to get the fidelity website')
        return output
    else:
        results_page = BeautifulSoup(response.content,'html5lib')
        # rating
        if len(results_page.find_all('div', class_="_l0ao8q")) == 0:
            rating = 'no rating'
        else:
            rating_sec = results_page.find_all('div', class_="_l0ao8q")[0]
            rating = rating_sec.find('span').get('aria-label')
        # coordinates
        coordinates = re.search(
            '"listing_lat":([0-9]{2}.[0-9]*),"listing_lng":(-[0-9]{2}.[0-9]*),', 
            response.text)
        
        lat = coordinates.group(1)
        lon = coordinates.group(2)
        # host
        superhost = re.search('"is_superhost":(.{1,5})', 
                              response.text).group(1)
        
        if len(results_page.find_all('span', class_="_fgdupie"))==0:
            response_rate = "no info"
        else:
            host = results_page.find_all('span', class_="_fgdupie")
            #response rate
            response_rate = float('nan')
            for host_ in host[::-1]:
                if 'Response rate' in host_.text:
                    response_rate = host_.text.split(':')[-1]
                    break
                
      
        if re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text) != None:
            accommodates = re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text).group(1)
        else:
            accommodates = re.search('"guest_label":"([0-9][0-9]?) guest.*', response.text).group(1)
        #print (accommodates)
        
        
        #bedrooms
        if re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text) != None:
            bedrooms = (re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text)).group(1)
        else:
            bedrooms = "studio"
            #print (bedrooms)
            
        #bathrooms
        try:
            bathrooms = (re.search('"bathroom_label":"([0-9]\.?[0-9]?).*","bed_label"', response.text)).group(1)
        except:
            bathrooms = re.search('"bathroom_label":"([0-9]\.?[0-9]?).*","bed_label"', response.text)
        #beds
        
        try:
            beds = (re.search('"bed_label":"(.).*","bedroom_label"', response.text)).group(1)
        except:
            beds = re.search('"bed_label":"(.).*","bedroom_label"', response.text)
        #print (beds)
        
        #room type
        try:
            room_type = results_page.find('span',class_="_1hh2h7tb").text
        except AttributeError:
            room_type='NoneType'

        #number of review
        try:
            number_of_reviews = results_page.find('div', class_="_vy3ibx").text
        except AttributeError:
            number_of_reviews='NoneType'
        
        output=[price, accommodates, bedrooms,beds, bathrooms, room_type, 
                number_of_reviews,  rating, lat, lon, superhost, response_rate]
        #print('output=')
        #print(output)
        
    return output

In [ ]:
#this function returns the urls for all the apartments listed on this main page

def get_prices_urls(url,dataframe):
    info_list=[]
    from selenium import webdriver  
    browser = webdriver.Chrome('/Users/daqiliu/Desktop/chromedriver')
    #url='https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&allow_override%5B%5D=&price_min=100&price_max=199&s_tag=qfon6p2A'
    browser.get(url)
    html_source = browser.page_source
    #browser.quit()
    #get the prices for every apartment since the price infomation is not availabe inside the apt page
    prices = browser.find_elements(By.XPATH,"//span[@class='_17oldnte']")
    price_list = []
    for price in prices:
        try:
            price = price.text
            price_list.append(price)
        except:
            pass
    price_list = price_list[::3]
    browser.quit()
    print("length of price list is: ", len(price_list))

    #get the url for every apt on this page
    results_page = BeautifulSoup(html_source, "lxml")
    
    #print (results_page)
    
    urls = results_page.find_all("meta", content=re.compile("www.airbnb.com/rooms"))
    #prices = (re.findall('"amount_formatted":"\$([0-9]{2,6})",', results_page.text))
    #print(prices)
    #urls = browser.find_elements(By.XPATH,"//a[@class='_1ol0z3h']")
    url_list = []
    for url in urls:
        url = str(url)
        url = "https://"+url[15:-18]
        #print (x)
        url_list.append(url)
    print("length of apt_url_list is:" + str(len(url_list)))
    print("-----------------")
    #print(url_list)
    #pass url and price to the get_house_info function and extract the rest infomation 
    
    if (len(price_list) == len(url_list)) and (len(price_list)>0):
        for x in range(len(price_list)):
            #print(get_house_info(url_list[x], price_list[x]))
            info_list.append(get_house_info(url_list[x], price_list[x]))
        #print('info list is:')
        #print(info_list)
        df2=pd.DataFrame(info_list,columns=['price','accommodates', 'bedrooms','beds', 
                                            'bathrooms', 'room_type', 'number_of_reviews',  'rating', 
                                            'lat', 'lon', 'superhost', 'response_rate'])
        #print('dataframe is:')
        #print(df2)
        
        dataframe=pd.concat([dataframe,df2], ignore_index=True)
        print('complete dataframe')

    #print("info_list=")
    #print(info_list)
    return dataframe  

In [ ]:
page_url="https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&adults=0&children=0&infants=0&guests=0&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&price_min=917&allow_override%5B%5D=&s_tag=eSRH0a1_"
url_list=get_url_list(page_url)
df = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 
                           'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 
                           'superhost', 'response_rate'])
for url in url_list:
    df=get_prices_urls(url,df)

In [ ]:
df.to_csv('airbnbdata.csv')

# Data Cleaning

In [3]:
import pandas as pd
import numpy as np
df_final = pd.read_csv('airbnbdata.csv').iloc[:,1:]
df_final = df_final.drop_duplicates()


In [4]:
import re
def extract_price(string):
    pattern = r'[$0-9,]+'
    if re.search(pattern, string):
        price_tmp = re.search(pattern, string).group(0)
        price = re.sub('[$,]','',price_tmp)
        return float(price)
    else:
        return None
    
def num_reviews(string):
    pattern = r'^\d+'
    if re.search(pattern, string):
        return int(re.search(pattern, string).group(0))
    else:
        return 0
    
def superhost(string):
    if string == 'FALSE':
        return 0
    else:
        return 1
    
def rating(string):
    pattern = r'[0-9.]+'
    if re.search(pattern, string):
        return float(re.search(pattern, string).group(0))
    else:
        return 0
    
    
popular_room_type = df_final.groupby('room_type').count()[
    df_final.groupby('room_type').count()['price']>50].iloc[[0,1,2,3,4,6]].index.tolist()

def room_type(string):
    if string not in popular_room_type:
        return 'others'
    else:
        return string


In [5]:
df_final['price'] = df_final['price'].apply(lambda p: extract_price(p))
df_final['number_of_reviews'] = df_final['number_of_reviews'].apply(lambda x: num_reviews(x))
df_final['superhost'] = df_final['superhost'].apply(lambda x:superhost(x))
df_final['rating'] = df_final['rating'].apply(lambda x: rating(x))
df_final['room_type'] = df_final['room_type'].apply(lambda x: room_type(x))

df_final = pd.get_dummies(df_final, columns = ['bedrooms'], prefix = ['bedrooms'])
df_final = pd.get_dummies(df_final, columns = ['room_type'], prefix = ['room_type'])

In [6]:
df_final.head(10)

,price,accommodates,beds,bathrooms,number_of_reviews,rating,lat,lon,superhost,response_rate,...,bedrooms_7,bedrooms_8,bedrooms_studio,room_type_Entire apartment,room_type_Entire condominium,room_type_Entire loft,room_type_Entire serviced apartment,room_type_Entire townhouse,room_type_Private room in apartment,room_type_others
0,49.0,2,1.0,1.0,554,4.5,40.822641,-73.940409,0,100%,...,0,0,0,0,0,0,0,0,1,0
1,51.0,2,2.0,1.0,134,4.5,40.791538,-73.942331,0,90%,...,0,0,0,0,0,0,0,0,1,0
2,47.0,2,1.0,1.0,220,4.5,40.799565,-73.964680,0,100%,...,0,0,0,0,0,0,0,0,1,0
3,53.0,2,2.0,1.0,78,4.0,40.791416,-73.946641,0,90%,...,0,0,0,0,0,0,0,0,1,0
4,33.0,2,1.0,1.0,33,4.0,40.783393,-73.951685,0,100%,...,0,0,1,0,0,0,0,0,1,0
5,39.0,5,3.0,2.0,125,5.0,40.794391,-73.946024,1,97%,...,0,0,0,0,0,0,0,0,0,1
6,50.0,6,3.0,1.0,183,4.5,40.790244,-73.940635,0,100%,...,0,0,0,0,0,0,0,0,1,0
7,42.0,3,1.0,1.0,86,4.5,40.712800,-73.989089,0,100%,...,0,0,0,0,0,0,0,0,1,0
8,34.0,4,2.0,1.0,262,4.5,40.810504,-73.925071,0,91%,...,0,0,0,0,0,0,0,0,1,0
9,49.0,4,2.0,1.0,100,4.5,40.793912,-73.944316,1,97%,...,0,0,0,0,0,0,0,0,1,0


In [7]:
df_new = df_final.drop(columns = ['response_rate','lat','lon'])
df_new = df_new.dropna(how = 'any', axis = 0)
df_new.head(8)

,price,accommodates,beds,bathrooms,number_of_reviews,rating,superhost,bedrooms_1,bedrooms_2,bedrooms_3,...,bedrooms_7,bedrooms_8,bedrooms_studio,room_type_Entire apartment,room_type_Entire condominium,room_type_Entire loft,room_type_Entire serviced apartment,room_type_Entire townhouse,room_type_Private room in apartment,room_type_others
0,49.0,2,1.0,1.0,554,4.5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,51.0,2,2.0,1.0,134,4.5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,47.0,2,1.0,1.0,220,4.5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,53.0,2,2.0,1.0,78,4.0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,33.0,2,1.0,1.0,33,4.0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
5,39.0,5,3.0,2.0,125,5.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,50.0,6,3.0,1.0,183,4.5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
7,42.0,3,1.0,1.0,86,4.5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


# Train Linear Regression Model

In [8]:
from sklearn.model_selection import train_test_split
X = df_new.iloc[:, 1:]
y = df_new['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, random_state=42)

In [9]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
pred_price = reg.predict(X_test)

In [12]:
X_test['pred_price'] = pred_price
X_test['actual_price'] = y_test
X_test

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,accommodates,beds,bathrooms,number_of_reviews,rating,superhost,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,...,bedrooms_studio,room_type_Entire apartment,room_type_Entire condominium,room_type_Entire loft,room_type_Entire serviced apartment,room_type_Entire townhouse,room_type_Private room in apartment,room_type_others,pred_price,actual_price
1955,4,1.0,1.0,0,0.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,572.878852,900.0
29,1,1.0,1.0,29,5.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,83.321199,45.0
210,2,1.0,2.0,4,5.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,295.007808,38.0
956,3,2.0,1.0,12,5.0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,257.615454,250.0
584,2,1.0,1.0,0,0.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,366.143783,85.0
1605,5,2.0,2.0,19,4.5,0,0,1,0,0,...,0,0,0,0,1,0,0,0,501.446064,399.0
1238,4,3.0,1.0,80,5.0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,207.967346,298.0
51,1,1.0,1.0,15,5.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,195.013271,42.0
673,5,3.0,1.0,78,4.0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,269.077670,100.0
652,2,1.0,1.0,24,5.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,213.454664,100.0


In [13]:
reg.coef_

array([  24.41670586,  -26.20522677,  170.67181265,   -0.66392364,
        -47.83041845,  -37.98222735, -189.44901132, -201.91365927,
       -255.03827945,  -41.34793826,  -41.56947812,  472.20157074,
       -154.85960553,  622.99304474, -211.01664353,  -89.45779626,
         16.95284653,  177.04231534,  -69.22272667,  278.55476158,
       -227.12438457,  -86.74501596])

In [16]:
reg.intercept_ 

589.4171808747718

In [17]:
df_new.columns[1:]

Index(['accommodates', 'beds', 'bathrooms', 'number_of_reviews', 'rating',
       'superhost', 'bedrooms_1', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4',
       'bedrooms_5', 'bedrooms_6', 'bedrooms_7', 'bedrooms_8',
       'bedrooms_studio', 'room_type_Entire apartment',
       'room_type_Entire condominium', 'room_type_Entire loft',
       'room_type_Entire serviced apartment', 'room_type_Entire townhouse',
       'room_type_Private room in apartment', 'room_type_others'],
      dtype='object')

In [ ]:
df_final.to_csv('df_preprocessed_data.csv')
df_new.to_csv('df_build_model.csv')

# GUI 

In [5]:
import tkinter as tk

from tkinter import *
from tk import *

window = tk.Tk()
window.title("Airbnb price recommendation")
window.geometry("800x800")


#this function takes the inputs from GUI and calculate the price
def calculate_price():
    
    #these coefficients come from the output of the linear regression
    intercept = 589.4171808747672
    C_accommodates = 24.41670586
    C_beds = -26.20522677
    C_bathrooms = 170.67181265
    C_number_of_reviews = -0.66392364
    C_rating = -47.83041845
    C_superhost = -37.98222735
    C_bedroom1 = -189.44901132
    C_bedroom2 = -201.91365927
    C_bedroom3 = -255.03827945
    C_bedroom4 = -41.34793826
    C_bedroom5 = -41.56947812
    C_bedroom6 = 472.20157074
    C_bedroom7 = -154.85960553
    C_bedroom8 = 622.99304474
    C_bedroomstudio = -211.01664353
    
    C_Entire_apartment = -89.45779626
    C_Entire_serviced_apartment = -69.22272667  
    C_Entire_condominium = 16.95284653
    C_Entire_loft = 177.04231534 
    C_Entire_townhouse = 278.55476158
    C_Private_room_in_apartment = -227.12438457
    C_others = -86.74501596
    
    
    accomodates = float(entry_field1.get())
    beds = float(entry_field3.get())
    bathrooms = float(entry_field4.get())
    number_of_review = float(entry_field5.get())
    super_host = float(entry_field6.get())
    rating = float(entry_field7.get())
    #dummy variables for bedrooms
    bedroom1 = 0
    bedroom2 = 0
    bedroom3 = 0
    bedroom4 = 0
    bedroom5 = 0
    bedroom6 = 0
    bedroom7 = 0
    bedroom8 = 0
    bedroomstudio = 0
    
    if tkvar.get() == "1":
        bedroom1 = 1
    elif tkvar.get() == "2":
        bedroom2 = 1
    elif tkvar.get() == "3":
        bedroom3 = 1
    elif tkvar.get() == "4":
        bedroom4 = 1
    elif tkvar.get() == "5":
        bedroom5 = 1
    elif tkvar.get() == "6":
        bedroom6 = 1
    elif tkvar.get() == "7":
        bedroom7 = 1
    elif tkvar.get() == "8":
        bedroom8 = 1
    elif tkvar.get() == "studio":
        bedroomstudio = 1
        
        

    #dummy variables for roomtype
    Entire_apartment = 0
    Entire_serviced_apartment = 0
    Entire_condominium = 0
    Entire_loft = 0
    Entire_townhouse = 0
    Private_room_in_apartment = 0
    others = 0
    
    
    if tkvar2.get() == 'Entire apartment':
        Entire_apartment = 1
    elif tkvar2.get() == 'Entire serviced apartment':
        Entire_serviced_apartment = 1
    elif tkvar2.get() == 'Entire condominium':
        Entire_condominium = 1
    elif tkvar2.get() == 'Entire loft':
        Entire_loft = 1
    elif tkvar2.get() == 'Entire townhouse':
        Entire_townhouse = 1
    elif tkvar2.get() == 'Private room in apartment':
        Private_room_in_apartment = 1
    elif tkvar2.get() == 'others':
        others = 1
    

  
    price = (intercept+accomodates*C_accommodates +beds*C_beds + bathrooms * C_bathrooms + C_number_of_reviews * number_of_review
    + super_host * C_superhost + rating * C_rating + bedroom1 * C_bedroom1 + bedroom2 * C_bedroom2 + bedroom3 * C_bedroom3
    + bedroom4 * C_bedroom4 + bedroom5 * C_bedroom5 + bedroom6 * C_bedroom6 + bedroom7 * C_bedroom7 
    + bedroom8 * C_bedroom8 +  bedroomstudio * C_bedroomstudio + Entire_apartment * C_Entire_apartment
    + Entire_serviced_apartment * C_Entire_serviced_apartment + Entire_condominium * C_Entire_condominium
    + Entire_loft * C_Entire_loft + Entire_townhouse * C_Entire_townhouse + others * C_others + Private_room_in_apartment*C_Private_room_in_apartment)
    #roomtype = entry_field5.get()
    #print (type(accomodates))
    return ("price =" + str(price))
 
def display_price():
    price = calculate_price()
    price_display = tk.Text(master = window, height=10, width = 30)
    price_display.grid(column = 0, row = 31)
    price_display.insert(tk.END,price)
    
    

#the code below creates the GUI
    

title = tk.Label(text="please input below information to get the recommendated price")
title.grid()

#accomodates
accomodates = tk.Label(text = "Input number of accomodates")
accomodates.grid(column = 0, row = 1)
entry_field1 = tk.Entry()
entry_field1.grid(column = 0, row = 2)


#bedrooms, this is a categorical variable
bedrooms = tk.Label(text = "Input number of bedrooms")
bedrooms.grid(column = 0, row = 3)
bedroom_number = { '1','2','3','4','5', '6','7','8','studio'}
tkvar = tk.StringVar()
tkvar.set('1') # set the default option
popupMenu = OptionMenu(window, tkvar, *bedroom_number)
popupMenu.grid(column = 0, row =4)



#number of beds
beds = tk.Label(text = "Input number of beds")
beds.grid(column = 0, row = 5)
entry_field3 = tk.Entry()
entry_field3.grid(column = 0, row = 6)


#number of bathrooms
bathrooms = tk.Label(text = "Input number of bathrooms")
bathrooms.grid(column = 0, row = 7)
entry_field4 = tk.Entry()
entry_field4.grid(column = 0, row = 8)

#number of reviews
reviews = tk.Label(text = "Input number of review")
reviews.grid(column = 0, row = 9)
entry_field5 = tk.Entry()
entry_field5.grid(column = 0, row = 10)



#number of superhost
superhost = tk.Label(text = "Input if the host is superhost, 1 for yes and 0 for no")
superhost.grid(column = 0, row = 11)
entry_field6 = tk.Entry()
entry_field6.grid(column = 0, row = 12)




#roomtype, this is a categorical variable
roomtype = tk.Label(text = "Input roomtype")
roomtype.grid(column = 0, row = 13)
roomtype_list = { 'Entire apartment','Entire serviced apartment','Entire condominium','Entire loft','Entire townhouse', 'Private room in apartment','others'}
tkvar2 = tk.StringVar()
tkvar2.set('Entire apartment') # set the default option
popupMenu2 = OptionMenu(window, tkvar2, *roomtype_list)
popupMenu2.grid(column = 0, row =14)

#rating 
rating = tk.Label(text = "Input rating, from 0 to 5")
rating.grid(column = 0, row = 15)
entry_field7 = tk.Entry()
entry_field7.grid(column = 0, row = 16)




calculate_button = tk.Button(text="calculate", command =display_price ) 
calculate_button.grid(column = 0, row = 30)


window.mainloop()

TclError: no display name and no $DISPLAY environment variable